In [1]:
import pandas as pd
import numpy as np
import matplotlib 
from matplotlib import pyplot as plt
from scipy.stats import ttest_ind, wilcoxon
from chord import Chord
import operator
#sklearn imports
from sklearn.decomposition import PCA #Principal Component Analysis
from sklearn.manifold import TSNE #T-Distributed Stochastic Neighbor Embedding
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler #used for 'Feature Scaling'
from sklearn.cluster import KMeans, SpectralClustering, AgglomerativeClustering
from sklearn.datasets import make_blobs
from sklearn import metrics
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.pyplot as plt
import matplotlib.cm as cm

#Basic imports
import numpy as np
import pandas as pd

#############################
# Princple Feature Analysis #
# PFA is based on http://venom.cs.utsa.edu/dmz/techrep/2007/CS-TR-2007-011.pdf #
#############################

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from collections import defaultdict
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.preprocessing import StandardScaler


In [2]:
#mdd_clf=pd.read_csv('/Users/xinxing/Documents/MDD/New data/MDD_test/data_new.csv').fillna(0)
mdd_clf=pd.read_excel('/Users/xinxing/Documents/MDD/New data/MDD_test/May_03_2021/MDD_data_Gaurav.xlsx')

In [3]:
mdd_clf

,7T ID,XNAT,Label,Status,Age,Gender,Height,Weight,BMI,Race,...,N_Right-VentralDC,N_Right-vessel,N_Right-choroid-plexus,N_WM-hypointensities,N_Optic-Chiasm,N_CC_Posterior,N_CC_Mid_Posterior,N_CC_Central,N_CC_Mid_Anterior,N_CC_Anterior
0,NT001,S02794,7409571,0,40,M,1.524000,88.450523,38.082940,5,...,3.804401,0.025932,0.532276,1.894469,0.092779,1.071083,0.342685,0.428260,0.397046,0.982338
1,NT002,S02855,7430705,0,49,F,1.625603,74.842751,28.321785,5,...,3.505706,0.054691,0.707489,2.695649,0.132873,0.918633,0.341327,0.434571,0.477965,1.151384
2,NT003,S03843,7747294,0,45,M,1.752604,87.543339,28.500715,3,...,3.607140,0.012102,0.370024,2.645058,0.190682,1.041461,0.633224,0.542124,0.558800,1.224615
3,NT004,S04179,7734800,0,51,M,1.905004,117.934031,32.497312,5,...,3.724191,0.017628,0.840838,4.193174,0.091664,0.845333,0.539141,0.481499,0.507147,1.005833
4,NT005,S03872,7759144,0,55,M,1.778004,68.038864,21.522455,5,...,3.597112,0.061952,0.627542,1.993024,0.120772,0.928297,0.395101,0.402050,0.427790,1.045936
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,NT122,S15671,1004381497,1,29,F,1.650999,52.163180,19.136849,7,...,3.661485,0.034242,0.446991,2.635848,0.053459,0.834493,0.583756,0.636704,0.557998,0.849825
84,NT123,S00554,1004802854,0,20,F,1.549399,56.699114,23.618354,5,...,3.641629,0.022027,0.446421,1.829644,0.069783,0.696501,0.327363,0.583804,0.541174,0.754891
85,NT124,S00696,1004912637,1,23,F,1.600199,97.522476,38.085236,1,...,3.392523,0.023829,0.387832,2.117238,0.126989,1.077888,0.632396,0.647694,0.620335,1.221842
86,NT125,S00846,1005027073,1,29,M,1.752599,63.503007,20.674221,2,...,3.696045,0.011396,0.510525,1.306817,0.141920,1.065580,0.659106,0.578373,0.647623,0.991070


In [4]:
import tableone
droplist=['XNAT','7T ID','Label','madrs_sum','qids_score','mddonset','mddc_duration', 'madrs_sum',
         'pss_score','btq_score','lsc_score','ce_tleq','oc_tleq','qids_score','rrs_total',
         'shaps_score_1','shaps_score_2','bss_total','sticsa_somatic','sticsa_cognitive','Race','Employment Status',
          'Household Income']
mdd_master=mdd_clf.drop(droplist,axis=1)
table = tableone.TableOne(data=mdd_master, 
                          groupby='Status', 
                          pval=True,
                          htest_name=True,
                          #smd = True,
                          sort='P-Value', 
                          label_suffix=True)

table.tabulate(tablefmt="html")
#table.to_excel('mdd_Pvalue_top20.xlsx')

,,Missing,Overall,0,1,P-Value,Test
n,,,88,42,46,,
"Rpostcentral, mean (SD)",,0,14.6 (1.2),14.3 (1.1),14.9 (1.1),0.005,Two Sample T-test
"LGcaudalanteriorcingulate, mean (SD)",,0,0.6 (0.1),0.6 (0.1),0.7 (0.2),0.006,Two Sample T-test
"RGpericalcarine, mean (SD)",,0,0.7 (0.1),0.7 (0.1),0.7 (0.1),0.009,Two Sample T-test
"LGfrontalpole, mean (SD)",,0,3.5 (0.6),3.3 (0.5),3.6 (0.6),0.014,Two Sample T-test
"N_Left-AN_CCumbens-area, mean (SD)",,0,0.5 (0.1),0.4 (0.1),0.5 (0.1),0.014,Two Sample T-test
"RGinferiorparietal, mean (SD)",,0,1.2 (0.1),1.2 (0.1),1.2 (0.1),0.024,Two Sample T-test
"LGinferiorparietal, mean (SD)",,0,1.2 (0.1),1.1 (0.1),1.2 (0.1),0.041,Two Sample T-test
"LGprecentral, mean (SD)",,0,1.0 (0.1),1.0 (0.1),1.0 (0.1),0.041,Two Sample T-test
"LGsuperiorparietal, mean (SD)",,0,1.0 (0.1),1.0 (0.1),1.0 (0.1),0.043,Two Sample T-test


In [5]:
y=mdd_master.Status
X=mdd_master.drop(['Age','Status','Gender','Height','Weight','BMI','Ethnicity','Education'],axis=1)

/Users/xinxing/opt/anaconda3/envs/myenv/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
import matplotlib.pyplot as plt

from sklearn import svm
from scipy import interp
import numpy as np
import pandas as pd
from itertools import cycle

from sklearn.ensemble import (RandomForestClassifier,
                              RandomForestRegressor)
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.utils import resample
from sklearn.utils import shuffle
from sklearn.preprocessing import label_binarize
from sklearn.metrics import f1_score
list20=['N_Right-Thalamus-Proper', 'Rpostcentral', 'LGsuperiorparietal',
       'N_Left-AN_CCumbens-area', 'LGentorhinal', 'LGcaudalanteriorcingulate',
       'N_Optic-Chiasm', 'RGinferiorparietal', 'N_Right-Inf-Lat-Vent',
       'N_Right-AN_CCumbens-area', 'Linferiorparietal', 'LGfrontalpole',
       'RGpericalcarine', 'RGpostcentral', 'Linsula', 'N_Right-vessel',
       'Lprecentral', 'Rsuperiorfrontal', 'LGprecentral', 'LGfusiform']
X=X[list20]
numFeature=X.shape[1]
#X=X.as_matrix()
y=y.ravel()
X_1=X
print(X_1.columns)


#normalized standardize features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
X=scaler.transform(X)


#clf=RandomForestClassifier(n_estimators=100,criterion='entropy')#,class_weight='balanced')
#clf=svm.SVC(kernel='rbf',C=1000, gamma=0.01,coef0=0.01,probability=True)
#cv = StratifiedKFold(n_splits=3)
tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 250)
result= []
importance=np.zeros((500,numFeature))
res=[]
f1=[]

i = 0
j=0 
for i in range(100):
    clf = RandomForestClassifier(n_estimators=300, min_samples_split=5, min_samples_leaf=8, max_features='auto',max_depth=50)
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=i)
    for train, test in cv.split(X, y):
    
        #sm= SMOTEENN(random_state=44)
        #X_res,y_res=sm.fit_sample(X[train],y[train])
        #probas_ = clf.fit(X_res, y_res).predict_proba(X[test])#The smote upsampling
        probas_ = clf.fit(X[train], y[train]).predict_proba(X[test]) #original
        accuracy=clf.predict(X[test])
        res.append(clf.score(X[test],y[test]))
        #print(('accuracy: ')+str(clf.score(X[test],y[test])))
        y_pred=clf.predict(X[test])
        f1.append(f1_score(y[test], y_pred, average='weighted'))
        #print(('F1 socre: ')+str(f1_score(y[test], y_pred, average='weighted')))
        importance[j,:]=clf.feature_importances_
        
        j += 1
    i += 1
b=np.mean(importance, axis=0)
feature_importances = pd.DataFrame(b,index = X_1.columns, columns=['importance']).sort_values('importance',ascending=False)    
print("The acc is: "+str(np.mean(res)))
print("The f1 is: " +str(np.mean(f1)))
print("The feature ranking is:")
print(feature_importances)
feature_importances.to_csv('HCvsMDD_RF_top20.csv')

/Users/xinxing/opt/anaconda3/envs/myenv/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Index(['N_Right-Thalamus-Proper', 'Rpostcentral', 'LGsuperiorparietal',
       'N_Left-AN_CCumbens-area', 'LGentorhinal', 'LGcaudalanteriorcingulate',
       'N_Optic-Chiasm', 'RGinferiorparietal', 'N_Right-Inf-Lat-Vent',
       'N_Right-AN_CCumbens-area', 'Linferiorparietal', 'LGfrontalpole',
       'RGpericalcarine', 'RGpostcentral', 'Linsula', 'N_Right-vessel',
       'Lprecentral', 'Rsuperiorfrontal', 'LGprecentral', 'LGfusiform'],
      dtype='object')
The acc is: 0.6956143790849673
The f1 is: 0.6879660264329729
The feature ranking is:
                           importance
N_Right-Thalamus-Proper      0.106035
Rpostcentral                 0.069922
LGsuperiorparietal           0.060927
LGentorhinal                 0.058851
N_Left-AN_CCumbens-area      0.056226
LGcaudalanteriorcingulate    0.051717
N_Optic-Chiasm               0.051498
RGinferiorparietal           0.048291
N_Right-Inf-Lat-Vent         0.046038
N_Right-AN_CCumbens-area     0.045897
Linferiorparietal            0.0457

In [7]:
feature_importances[:10].index

/Users/xinxing/opt/anaconda3/envs/myenv/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Index(['N_Right-Thalamus-Proper', 'Rpostcentral', 'LGsuperiorparietal',
       'LGentorhinal', 'N_Left-AN_CCumbens-area', 'LGcaudalanteriorcingulate',
       'N_Optic-Chiasm', 'RGinferiorparietal', 'N_Right-Inf-Lat-Vent',
       'N_Right-AN_CCumbens-area'],
      dtype='object')